## Import The Libaries

In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import timedelta

## Import Data

In [2]:
df=pd.read_csv("Online Retail.csv")

In [3]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom


In [4]:
#dropping rows with null customerID
df.dropna(subset=["CustomerID"],inplace=True)
#Create new column for total price
df["Total_Price"]=df["Quantity"]*df["UnitPrice"]
#Taking only customers from united kingdom
df1=df[df["Country"]=="United Kingdom"]
df1.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total_Price
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom,20.34


In [5]:
#Conveting Date from str to datetime
df1["InvoiceDate"]=pd.to_datetime(df1["InvoiceDate"])
#adding one data to the last date
last_date=df1["InvoiceDate"].max()+timedelta(days=1)
df1["Date"]=df1["InvoiceDate"].apply(lambda x:(last_date-x).days)
#converting the format of the data
df1["InvoiceDate"]=df1["InvoiceDate"].apply(lambda x:datetime.datetime.strftime(x, '%d-%m-%Y'))

C:\Users\Gokul Mahendran\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Gokul Mahendran\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\Gokul Mahendran\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pan

## Recency

In [6]:
#Taking unique customerID rows based on thier last appearance on the store.
recency_table=df1[["CustomerID","Date"]].drop_duplicates(subset=["CustomerID"],keep="last")
recency_table.columns=["CustomerID","Recent_Days"]
recency_table.head()

,CustomerID,Recent_Days
167,18074.0,698
252,13747.0,698
540,17908.0,698
701,16583.0,698
865,17968.0,698


## Frequency

In [7]:
#Taking number of times a customer visited and bought something:
frequency_table=df1[["CustomerID","InvoiceDate"]].drop_duplicates()["CustomerID"].value_counts()
frequency_table=frequency_table.reset_index(drop=False)
frequency_table.columns=["CustomerID","Freq_Count"]
frequency_table.head()

## Monetary

In [12]:
#Taking the sum of monetery value each customer has made:
monetory_table=df1[["CustomerID","Total_Price"]].groupby(by="CustomerID",as_index=False).sum()
monetory_table.columns=["CustomerID","Total_Monetory"]
monetory_table.head()

In [17]:
#merging the tables:
final_table=pd.merge(recency_table[["CustomerID","Recent_Days"]],frequency_table[["CustomerID","Freq_Count"]],how="left")
final_table=pd.merge(final_table,monetory_table[["CustomerID","Total_Monetory"]],how="left")
final_table=final_table.sort_values(by="CustomerID").reset_index(drop=True)
final_table.head()

In [21]:
#Getting rfm score for each customer using qcut:
final_table["R"]=pd.qcut(final_table["Recent_Days"],3,labels=np.arange(3,0,-1))
final_table["F"]=pd.qcut(final_table["Freq_Count"],3,labels=np.arange(1,4))
final_table["M"]=pd.qcut(final_table["Total_Monetory"],3,labels=np.arange(1,4))
final_table.head()

In [25]:
#Summing the R,F,M score:
def join_score(x): return str(x['R']) + str(x['F']) + str(x['M'])
final_table['RFM_Segment'] = final_table.apply(join_score, axis=1)
final_table["RFM_Score"]=final_table[["R","F","M"]].sum(axis=1)
final_table.head()

In [31]:
customer_segments={9:"Perfect",
8:"Loyal",
7:"Can't Loose Them",
6:"Promising",
5:"Potential",
4:"Needs Attention",
3:"Competition_Takeaway"}

In [32]:
final_table["Customer_Segment"]=final_table["RFM_Score"].apply(lambda x:customer_segments[x])

In [33]:
final_table.head()

,CustomerID,Recent_Days,Freq_Count,Total_Monetory,R,F,M,RFM_Segment,RFM_Score,Customer_Segment
0,12346.0,327,1,0.00,1,1,1,111,3.0,Competition_Takeaway
1,12747.0,152,11,4196.01,2,3,3,233,8.0,Loyal
2,12748.0,90,115,29072.10,2,3,3,233,8.0,Loyal
3,12749.0,182,7,3868.20,2,3,3,233,8.0,Loyal
4,12820.0,182,4,942.34,2,2,2,222,6.0,Promising


In [30]:
final_table.groupby("RFM_Score").agg({
    'Recent_Days': ['mean',"median"],
    'Freq_Count': ['mean'],
    'Total_Monetory': ['mean',"median",'count']
}).round(1)

Recent_Days        Freq_Count Total_Monetory              
                 mean median       mean           mean  median count
RFM_Score                                                           
3.0             303.1  271.0        1.2          178.6   178.5   541
4.0             183.3  172.0        1.5          337.7   307.8   663
5.0             103.9   74.0        1.8          508.4   372.4   706
6.0             101.2   72.5        2.7          833.2   657.6   536
7.0             138.2  111.0        6.4         2783.5  1302.7   598
8.0              94.9   91.0       10.6         4934.2  2083.4   523
9.0              23.8   19.0        9.4         3646.6  2457.0   383